# Paso a parquet el archivo de contratos sin fechas y desordenado

In [1]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
conf = (SparkConf().set("spark.executor.memory", "8g")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ContratosAParque").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/22 17:41:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import substring

esquema = StructType() \
      .add("ctId",StringType(),True) \
      .add("fecRef",StringType(),True) \
      .add("dato",StringType(),True) \
      .add("saldo",IntegerType(),True)

df = spark.read.csv("./datos/fakeContracts01.csv", sep=";", header = True, schema = esquema)
print(df.count())
df = df.withColumn("proc", substring("ctId",1,6))
df.show()
df.orderBy('ctId', 'fecRef')
df.write.mode("overwrite").partitionBy("proc").parquet("./datos/fakeContractsSinFechas.parquet")

1276940
+------------+----------+--------------------+-----+------+
|        ctId|    fecRef|                dato|saldo|  proc|
+------------+----------+--------------------+-----+------+
|CT0001123484|2021-06-21|Dato del contrato...|47900|CT0001|
|CT0001123484|2021-08-19|Dato del contrato...| 6200|CT0001|
|CT0001123484|2021-04-18|Dato del contrato...| 2400|CT0001|
|CT0001123484|2020-11-21|Dato del contrato...| 1900|CT0001|
|CT0001123484|2021-04-01|Dato del contrato...|42000|CT0001|
|CT0001123484|2021-04-09|Dato del contrato...|33700|CT0001|
|CT0001123484|2021-07-20|Dato del contrato...|26500|CT0001|
|CT0001123484|2020-08-08|Dato del contrato...| 1800|CT0001|
|CT0001123484|2021-08-04|Dato del contrato...|43100|CT0001|
|CT0001123515|2021-06-06|Dato del contrato...|34800|CT0001|
|CT0001123515|2021-04-28|Dato del contrato...| 5200|CT0001|
|CT0001123515|2020-04-02|Dato del contrato...|70200|CT0001|
|CT0001123515|2020-03-28|Dato del contrato...|90400|CT0001|
|CT0001123515|2021-02-09|Dato de

In [3]:
spark.sparkContext.stop()